<a href="https://colab.research.google.com/github/mark-bell-tna/ComputationalAccess/blob/main/UKGWA_AtoZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests;      #used for connecting to the API
import sys
from time import sleep
from math import log
import os
from urllib.request import urlopen
import re
from operator import itemgetter

In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
print("Running in colab?:",RunningInCOLAB)
print("iPython string:",get_ipython())

In [4]:
!pip install beautifulsoup4

In [2]:
!git clone https://github.com/mark-bell-tna/ComputationalAccess.git
sys.path.insert(0,'ComputationalAccess')

Cloning into 'ComputationalAccess'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 43 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [3]:
from ukgwa_index import UKGWAIndex
from text_utils import SuffixTree, text_to_parts

In [ ]:
# Get entries from AtoZ index
idx = UKGWAIndex()
print("Getting index...")
idx.indexfromweb()
#idx.indexfromfile("atoz_index.txt")
#idx.indextofile("atoz_index.txt")

Getting index...


In [ ]:
# Index the text of the index to make it searchable and to get common ngrams

st = SuffixTree(0, set(['on','for','and','of','&','the']))

print("Loading suffixes...")
for w in idx:
    w_p = text_to_parts(w[1])
    i = 0
    for p in w_p:
        if p[3] == "web":
            continue
        i += 1
        st.add_tokens(p[0].split(" "), w[0] + "." + str(i))




Loading suffixes...


In [ ]:
# Print examples of top N common ngrams

print("Getting phrases...")
s = st.get_phrases(30,2,10)
s.sort(key=itemgetter(1), reverse=True)

# TODO: Need to incorporate nearest match code to get references
# Do inside suffix tree class
N = 5
for row in s[0:N]:
    print(" ".join(row[0]),row[1])
    print("\t",idx.indexlookup(st.get_ngram_reference(row[2])[:-2]))
    #for r in row[4]:
    #    print("\t",idx.indexlookup(st.get_ngram_reference(r)[:-2]))

Getting phrases...
Twitter Archive 340
	 ['UKGWA.13', 'ACAS (Advisory, Conciliation and Arbitration Service) – Twitter Archive', 'twitter', 'acasorguk', 'N']
YouTube Archive 194
	 ['UKGWA.14', 'ACAS (Advisory, Conciliation and Arbitration Service) – YouTube Archive', 'video', 'acasorguk', 'N']
National Health Service 170
	 ['UKGWA.1392', 'Department of Health (DH) – Innovation in the National Health Service (NHS)', '*', 'http://innovation.dh.gov.uk/', 'N']
GOV.UK Blog 143
	 ['UKGWA.5', 'Academies and free schools – GOV.UK Blog', '*', 'https://academyschools.blog.gov.uk/', 'N']
Advisory Committee 58
	 ['UKGWA.32', 'Administration of Radioactive Substances Advisory Committee', '*', 'http://www.arsac.org.uk/', 'N']
